In [1]:
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *

In [33]:
n_instances = 1000
subj_docs = [(sent, 'subj') for sent in subjectivity.sents(categories='subj')[:n_instances]]
obj_docs = [(sent, 'obj') for sent in subjectivity.sents(categories='obj')[:n_instances]]
len(subj_docs), len(obj_docs)

(1000, 1000)

In [65]:
subj_docs

[([u'smart',
   u'and',
   u'alert',
   u',',
   u'thirteen',
   u'conversations',
   u'about',
   u'one',
   u'thing',
   u'is',
   u'a',
   u'small',
   u'gem',
   u'.'],
  'subj'),
 ([u'color',
   u',',
   u'musical',
   u'bounce',
   u'and',
   u'warm',
   u'seas',
   u'lapping',
   u'on',
   u'island',
   u'shores',
   u'.',
   u'and',
   u'just',
   u'enough',
   u'science',
   u'to',
   u'send',
   u'you',
   u'home',
   u'thinking',
   u'.'],
  'subj'),
 ([u'it',
   u'is',
   u'not',
   u'a',
   u'mass-market',
   u'entertainment',
   u'but',
   u'an',
   u'uncompromising',
   u'attempt',
   u'by',
   u'one',
   u'artist',
   u'to',
   u'think',
   u'about',
   u'another',
   u'.'],
  'subj'),
 ([u'a',
   u'light-hearted',
   u'french',
   u'film',
   u'about',
   u'the',
   u'spiritual',
   u'quest',
   u'of',
   u'a',
   u'fashion',
   u'model',
   u'seeking',
   u'peace',
   u'of',
   u'mind',
   u'while',
   u'in',
   u'a',
   u'love',
   u'affair',
   u'with',
   u'a',
   

In [34]:
subj_docs[0]

([u'smart',
  u'and',
  u'alert',
  u',',
  u'thirteen',
  u'conversations',
  u'about',
  u'one',
  u'thing',
  u'is',
  u'a',
  u'small',
  u'gem',
  u'.'],
 'subj')

In [37]:
train_subj_docs = subj_docs[:800]

In [38]:
test_subj_docs = subj_docs[800:1000]

In [39]:
train_obj_docs = obj_docs[:800]

In [40]:
test_obj_docs = obj_docs[800:1000]

In [41]:
training_docs = train_subj_docs+train_obj_docs

In [42]:
testing_docs = test_subj_docs+test_obj_docs

In [43]:
sentim_analyzer = SentimentAnalyzer()

In [44]:
all_words_neg = sentim_analyzer.all_words([mark_negation(doc) for doc in training_docs])

In [45]:
unigram_feats = sentim_analyzer.unigram_word_feats(all_words_neg, min_freq=4)

In [46]:
len(unigram_feats)

915

In [47]:
sentim_analyzer.add_feat_extractor(extract_unigram_feats, unigrams=unigram_feats)

In [48]:
training_set = sentim_analyzer.apply_features(training_docs)

In [49]:
test_set = sentim_analyzer.apply_features(testing_docs)

In [50]:
trainer = NaiveBayesClassifier.train

In [51]:
classifier = sentim_analyzer.train(trainer, training_set)

Training classifier


In [52]:
for key,value in sorted(sentim_analyzer.evaluate(test_set).items()):
    print('{0}: {1}'.format(key, value))

Evaluating NaiveBayesClassifier results...
Accuracy: 0.835
F-measure [obj]: 0.826315789474
F-measure [subj]: 0.842857142857
Precision [obj]: 0.872222222222
Precision [subj]: 0.804545454545
Recall [obj]: 0.785
Recall [subj]: 0.885


In [53]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [54]:
sentences = ["VADER is smart, handsome, and funny.", # positive sentence example
                "VADER is smart, handsome, and funny!", # punctuation emphasis handled correctly (sentiment intensity adjusted)
                "VADER is very smart, handsome, and funny.",  # booster words handled correctly (sentiment intensity adjusted)
                "VADER is VERY SMART, handsome, and FUNNY.",  # emphasis for ALLCAPS handled
                "VADER is VERY SMART, handsome, and FUNNY!!!",# combination of signals - VADER appropriately adjusts intensity
                "VADER is VERY SMART, really handsome, and INCREDIBLY FUNNY!!!",# booster words & punctuation make this close to ceiling for score
                "The book was good.",         # positive sentence
                "The book was kind of good.", # qualified positive sentence is handled correctly (intensity adjusted)
                "The plot was good, but the characters are uncompelling and the dialog is not great.", # mixed negation sentence
                "A really bad, horrible book.",       # negative sentence with booster words
                "At least it isn't a horrible book.", # negated negative sentence with contraction
                ":) and :D",     # emoticons handled
                "",              # an empty string is correctly handled
                "Today sux",     #  negative slang handled
                "Today sux!",    #  negative slang with punctuation emphasis handled
                "Today SUX!",    #  negative slang with capitalization emphasis
                "Today kinda sux! But I'll get by, lol" # mixed sentiment example with slang and constrastive conjunction "but"
            ]

In [55]:
paragraph = "It was one of the worst movies I've seen, despite good reviews. \
Unbelievably bad acting!! Poor direction. VERY poor production. \
The movie was bad. Very bad movie. VERY bad movie. VERY BAD movie. VERY BAD movie!"

In [56]:
from nltk import tokenize

In [57]:
lines_list = tokenize.sent_tokenize(paragraph)

In [58]:
sentences.extend(lines_list)

In [59]:
tricky_sentences = [
    "Most automated sentiment analysis tools are shit.",
    "VADER sentiment analysis is the shit.",
    "Sentiment analysis has never been good.",
    "Sentiment analysis with VADER has never been this good.",
    "Warren Beatty has never been so entertaining.",
    "I won't say that the movie is astounding and I wouldn't claim that \
    the movie is too banal either.",
    "I like to hate Michael Bay films, but I couldn't fault this one",
    "It's one thing to watch an Uwe Boll film, but another thing entirely \
    to pay for it",
    "The movie was too good",
    "This movie was actually neither that funny, nor super witty.",
    "This movie doesn't care about cleverness, wit or any other kind of \
    intelligent humor.",
    "Those who find ugly meanings in beautiful things are corrupt without \
    being charming.",
    "There are slow and repetitive parts, BUT it has just enough spice to \
    keep it interesting.",
    "The script is not fantastic, but the acting is decent and the cinematography \
    is EXCELLENT!",
    "Roger Dodger is one of the most compelling variations on this theme.",
    "Roger Dodger is one of the least compelling variations on this theme.",
    "Roger Dodger is at least compelling as a variation on the theme.",
    "they fall in love with the product",
    "but then it breaks",
    "usually around the time the 90 day warranty expires",
    "the twin towers collapsed today",
    "However, Mr. Carter solemnly argues, his client carried out the kidnapping \
    under orders and in the ''least offensive way possible.''"
]

In [60]:
sentences.extend(tricky_sentences)

In [61]:
sid = SentimentIntensityAnalyzer()

In [62]:
for sentence in sentences:
    print(sentence)
    ss = sid.polarity_scores(sentence)
    for k in sorted(ss):
        print('{0}: {1}, '.format(k, ss[k], end=''))
    print()

VADER is smart, handsome, and funny.
compound: 0.8316, 
neg: 0.0, 
neu: 0.254, 
pos: 0.746, 
()
VADER is smart, handsome, and funny!
compound: 0.8439, 
neg: 0.0, 
neu: 0.248, 
pos: 0.752, 
()
VADER is very smart, handsome, and funny.
compound: 0.8545, 
neg: 0.0, 
neu: 0.299, 
pos: 0.701, 
()
VADER is VERY SMART, handsome, and FUNNY.
compound: 0.9227, 
neg: 0.0, 
neu: 0.246, 
pos: 0.754, 
()
VADER is VERY SMART, handsome, and FUNNY!!!
compound: 0.9342, 
neg: 0.0, 
neu: 0.233, 
pos: 0.767, 
()
VADER is VERY SMART, really handsome, and INCREDIBLY FUNNY!!!
compound: 0.9469, 
neg: 0.0, 
neu: 0.294, 
pos: 0.706, 
()
The book was good.
compound: 0.4404, 
neg: 0.0, 
neu: 0.508, 
pos: 0.492, 
()
The book was kind of good.
compound: 0.3832, 
neg: 0.0, 
neu: 0.657, 
pos: 0.343, 
()
The plot was good, but the characters are uncompelling and the dialog is not great.
compound: -0.7042, 
neg: 0.327, 
neu: 0.579, 
pos: 0.094, 
()
A really bad, horrible book.
compound: -0.8211, 
neg: 0.791, 
neu: 0.209

In [68]:
news='''
Apple Inc and Alphabet Inc's Google engage in practices that undermine competition in the smartphone app market, an advisory panel to Japan's Ministry of Economy, Trade and Industry alleged on Thursday.
Two of the practices it identified, restricting payment methods developers can accept and preventing the use of virtual currencies between apps, could violate Japan's anti-monopoly law, the advisory panel said in a report.
"We should monitor the situation," the report said. "If the competition regulator judges certain actions to be illegal, they should take firm legal measures in response."
Apple and Google officials were not immediately available for comment.
Anti-trust violations have become a sensitive issue for U.S. technology companies after the European Union accused Google in April of using its Android mobile operating system to block competitors, which could carry a fine of up to $7.4 billion.
The METI panel report looked at how the two U.S. technology giants, as well as others that control platforms through which smartphone applications are sold, use their positions of power to decide what app developers can and cannot do.
Japan's government has placed renewed focus on the technology industry after Prime Minister Shinzo Abe made investment in cutting edge technology a central part of his economic growth strategy.
(Reporting by Stanley White and Makiko Yamazaki in Tokyo, with Aishwarya Venugopal in Bengaluru; Editing by Christian Schmollinger)
'''

In [70]:
ss = sid.polarity_scores(news)
for k in sorted(ss):
        print('{0}: {1}, '.format(k, ss[k], end=''))

compound: -0.8225, 
neg: 0.093, 
neu: 0.844, 
pos: 0.062, 


In [71]:
news='''
Nov 15 Tidewater Midstream and Infrastructure Ltd
* Tidewater Midstream and Infrastructure - Tidewater expects Q4 2016 adjusted EBITDA to be in line with previously announced guidance of about $11 - $12 million
* Q3 earnings per share c$0.01
* Qtrly total revenues c$ 27 million versus c$ 7.9 million
* Q3 earnings per share view c$0.02 -- Thomson Reuters I/B/E/S
* Tidewater Midstream and Infrastructure Ltd. announces third quarter 2016 results Source text for 
'''

In [72]:
ss = sid.polarity_scores(news)
for k in sorted(ss):
        print('{0}: {1}, '.format(k, ss[k], end=''))

compound: 0.5267, 
neg: 0.0, 
neu: 0.937, 
pos: 0.063, 


In [75]:
news='''
Iceberg Research on Wednesday said it expects construction company Tutor Perini Corp's share price to tumble as much as 55 percent, hurt by weak financials and weak corporate governance plus concerns whether planned infrastructure spending will become a reality.
In a report issued on Wednesday and seen by Reuters, Iceberg said shares could tumble to $13 a share. It fell 6 percent to trade at $27.05 in early trading on Wednesday.
Like other construction companies, Tutor Perini's stock surged after Donald Trump won the U.S. presidential race and has called for heavy infrastructure spending.
But Iceberg said Tutor Perini, with a $1.5 billion market capitalization, faces net debt of $622 million at a time when banks are cutting their exposure to the company. It also noted that investors have complained about weak governance where their votes on pay have been ignored for years.
And it said Trump's planned infrastructure spending is not a sure thing. Since the election, the stock price has jumped 35 percent.
"Interest expenses will plague Tutor’s profitability compared to its peers," the report said.
"Going forward Tutor may be engaged in a vicious cycle: its declining financial strength may prevent the company from bidding on some contracts or force the company to lower its prices even more aggressively," the Iceberg Research report, seen by Reuters, says. "Our target price for Tutor’s stock is $13 per share (55 percent downside)," the report added.
'''

In [76]:
ss = sid.polarity_scores(news)
for k in sorted(ss):
        print('{0}: {1}, '.format(k, ss[k], end=''))

compound: -0.2644, 
neg: 0.112, 
neu: 0.785, 
pos: 0.104, 
